In [1]:
# Avoids reloading kernel while developing
%load_ext autoreload
%autoreload 2

In [2]:
from PyCorrelationMatrixManager.correlation_matrix import *

import os

from src.utilities import *
from src.print_utilities import *

#pretty latex printing
from IPython.display import display, Math 
pprint = lambda o : display(Math(str(o)))



## Operators

Operators for SU(4) baryons are constructed in full at [su4ops](github.com/chrisculver/su4ops) or a repository of Kimmy Cushman.  Here they are translated into the correct objects for the WickContraction library.  Specifically at the moment the below code sets up a list of "Elemental Operators" in different irreps, with a gamma matrix name and spatial coordinate.  

In [3]:
# load definition of the building blocks of the operators.
from src.udud_elementals import *
import numpy as np

from WickContractions.laph.diagram import *

#gammas = [('buchoff',0)]
gammas=[]
gammas += np.array([[('A1g',i) for i in range(1)] for row in range(1)]).reshape(1,2).tolist()
gammas += np.array([[('T1g{}'.format(row),i) for i in range(1)] for row in range(1)]).reshape(1,2).tolist()
gammas += np.array([[('Eg{}'.format(row),i) for i in range(2)] for row in range(2)]).reshape(4,2).tolist()

print(gammas)

gammaName = lambda irrep,i : "\\Gamma^{{ ({},{}) }}".format(irrep, i)

aOps=[] # annihilation operators
cOps=[] # creation operators
for g in gammas:
    aOps.append(Operator([baryonSink(1,['u','u','u','u'],'p_0',gammaName(g[0],g[1]))]))
    cOps.append(Operator([baryonSource(1,['u','u','u','u'], 'p_1',gammaName(g[0],g[1]))]))
    
    #aOps.append(Operator([baryonSink([1,-1,+1,0.5],[['d','u','u','u'],['u','d','u','u'],...],'x_0',gammaName(g[0],g[1]))]))

[['A1g', '0'], ['T1g0', '0'], ['Eg0', '0'], ['Eg0', '1'], ['Eg1', '0'], ['Eg1', '1']]


In [4]:
pprint(aOps[0])

<IPython.core.display.Math object>

In [5]:
pprint(cOps[0])

<IPython.core.display.Math object>

## Wick Contractions

Now for each combination of annihilation and creation operator we compute the wick contractions, and store every diagram in a list.  The example diagram printed out is the full expression with all indices and dependencies for non-momentum projected baryons.

In [6]:
corrMatrix = CorrelationMatrix(cOps, aOps)
corrMatrix.contract()
corrMatrix.laphify()

Now we convert the above diagram into a LapH version, in terms of a baryon source function and baryon propagator.  These are defined in the "SU4_Correlators" tex file in the LSD overleaf.

In [21]:
allBaryonTensors, allBaryonSinks, allBaryonProps = corrMatrix.get_baryon_tensor_dictionaries()

In [22]:
allBaryonTensors

{'B(p_1,t_i,\\Gamma^{ (A1g,0) })': 0,
 'B^*(p_0,t_i,t_f,\\Gamma^{ (A1g,0) })': 1,
 'B^*(p_0,t_i,t_f,\\Gamma^{ (T1g0,0) })': 2,
 'B^*(p_0,t_i,t_f,\\Gamma^{ (Eg0,0) })': 3,
 'B^*(p_0,t_i,t_f,\\Gamma^{ (Eg0,1) })': 4,
 'B^*(p_0,t_i,t_f,\\Gamma^{ (Eg1,0) })': 5,
 'B^*(p_0,t_i,t_f,\\Gamma^{ (Eg1,1) })': 6,
 'B(p_1,t_i,\\Gamma^{ (T1g0,0) })': 7,
 'B(p_1,t_i,\\Gamma^{ (Eg0,0) })': 8,
 'B(p_1,t_i,\\Gamma^{ (Eg0,1) })': 9,
 'B(p_1,t_i,\\Gamma^{ (Eg1,0) })': 10,
 'B(p_1,t_i,\\Gamma^{ (Eg1,1) })': 11}

In [23]:
allBaryonProps

{'B^*(p_0,t_i,t_f,\\Gamma^{ (A1g,0) })': 0,
 'B^*(p_0,t_i,t_f,\\Gamma^{ (T1g0,0) })': 1,
 'B^*(p_0,t_i,t_f,\\Gamma^{ (Eg0,0) })': 2,
 'B^*(p_0,t_i,t_f,\\Gamma^{ (Eg0,1) })': 3,
 'B^*(p_0,t_i,t_f,\\Gamma^{ (Eg1,0) })': 4,
 'B^*(p_0,t_i,t_f,\\Gamma^{ (Eg1,1) })': 5}

In [24]:
allBaryonSinks

{'B(p_1,t_i,\\Gamma^{ (A1g,0) })': 0,
 'B(p_1,t_i,\\Gamma^{ (T1g0,0) })': 1,
 'B(p_1,t_i,\\Gamma^{ (Eg0,0) })': 2,
 'B(p_1,t_i,\\Gamma^{ (Eg0,1) })': 3,
 'B(p_1,t_i,\\Gamma^{ (Eg1,0) })': 4,
 'B(p_1,t_i,\\Gamma^{ (Eg1,1) })': 5}

## Evaluating Diagrams

Now we generate a cpp file which contracts all of the rank 4 baryon tensors into diagrams.  This is to be used with [lapH_build_and_run](https://gitlab.com/Kimmy.Cushman/laph_build_and_run) on lassen.  Specifically the "Output/define_diagrams_gpu.cpp" file needs to be placed in the src folder of the baryons branch of that directory along with the generated(todo) input file.

In [25]:
create_diagram_gpu_file(corrMatrix.get_all_diagrams(), allBaryonTensors, allBaryonSinks, allBaryonProps)    
create_input_file(str(len(corrMatrix.get_all_diagrams())),gammas)
create_diagram_names_file(corrMatrix.get_all_diagrams())

TODO: Need to handle momenta and baryon flavor!


### Python code to combine diagrams

This is just a quick hack for a python script in the lapH_build_and_run directory to get correlators out of the output files.

In [15]:
diagrams=contract(aOps[0], cOps[0]).diagrams

In [16]:
laphDiagrams = [LDiagram(d) for d in diagrams]
simplify_laphDiagrams(laphDiagrams)

In [17]:
s=""
for i,d in enumerate(laphDiagrams):
    s+="+{}*allDiags[i+{},dt,t]".format(d.coef,i)
print(s)

+1.0*allDiags[i+0,dt,t]+-1.0*allDiags[i+1,dt,t]+1.0*allDiags[i+2,dt,t]+-1.0*allDiags[i+3,dt,t]+1.0*allDiags[i+4,dt,t]+-1.0*allDiags[i+5,dt,t]+1.0*allDiags[i+6,dt,t]+-1.0*allDiags[i+7,dt,t]+1.0*allDiags[i+8,dt,t]+-1.0*allDiags[i+9,dt,t]+1.0*allDiags[i+10,dt,t]+-1.0*allDiags[i+11,dt,t]+1.0*allDiags[i+12,dt,t]+-1.0*allDiags[i+13,dt,t]+1.0*allDiags[i+14,dt,t]+-1.0*allDiags[i+15,dt,t]+1.0*allDiags[i+16,dt,t]+-1.0*allDiags[i+17,dt,t]+1.0*allDiags[i+18,dt,t]+-1.0*allDiags[i+19,dt,t]+1.0*allDiags[i+20,dt,t]+-1.0*allDiags[i+21,dt,t]+1.0*allDiags[i+22,dt,t]+-1.0*allDiags[i+23,dt,t]
